In [57]:
!pip install transformers requests


In [58]:
import requests
from collections import deque


In [59]:
# Short-term memory (last 5 messages)
short_memory = deque(maxlen=5)

# Long-term memory (entire session)
long_memory = []


In [60]:
# Cache for repeated questions
cache = {}


In [61]:
#PRIMARY_MODEL = "HuggingFaceH4/zephyr-7b-beta"
#FALLBACK_MODEL = "HuggingFaceH4/zephyr-7b-beta"

In [62]:
def query_model(prompt, model=None):
    prompt_lower = prompt.lower()
    last_line = prompt.split("\n")[-1].lower()

    # 1️⃣ Memory question FIRST
    if "what did i say first" in prompt_lower:
        for msg in short_memory:
            if msg.startswith("User:"):
                return f"You first said: {msg.replace('User:', '').strip()}"
        return "I don't remember yet."

    # 2️⃣ How are you
    if "how are you" in last_line:
        return "I'm doing well! Thanks for asking 😊"

    # 3️⃣ Greeting (ONLY check latest input)
    if "hi" in last_line or "hello" in last_line:
        return "Hello! 👋 How can I help you today?"

    # 4️⃣ Default fallback
    return "I understand. Tell me more."


In [63]:
def build_context(user_input):
    context = "Conversation so far:\n"
    for msg in short_memory:
        context += msg + "\n"
    context += f"\nUser: {user_input}\nAssistant:"
    return context


In [64]:
def chatbot(user_input):

    # Cache check
    if user_input in cache:
        return cache[user_input]

    context = build_context(user_input)

    try:
        reply = query_model(context, PRIMARY_MODEL)
    except:
        reply = query_model(context, FALLBACK_MODEL)

    # Update memory
    short_memory.append(f"User: {user_input}")
    short_memory.append(f"Assistant: {reply}")
    long_memory.append((user_input, reply))

    cache[user_input] = reply
    return reply


In [65]:
print("Context-Aware Chatbot (type 'exit' to stop)")

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        break
    print("Bot:", chatbot(user_input))


Context-Aware Chatbot (type 'exit' to stop)
You: Hi
Bot: I understand. Tell me more.
You: How are you?
Bot: I understand. Tell me more.
You: What did I say first?
Bot: You first said: Hi
You: exit
